<a href="https://colab.research.google.com/github/Jade-Isimbi/Prediction-Pipeline/blob/main/Database_Prediction_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymongo sqlalchemy kaggle pandas matplotlib plotly graphviz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 19.0 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from google.colab import files
import json

# Upload your kaggle.json file
print("=" * 60)
print("STEP 1: Upload your kaggle.json file")
print("=" * 60)
print("Go to: https://www.kaggle.com/settings/account")
print("Scroll down to 'API' section and click 'Create New Token'")
print("This will download kaggle.json")
print("\nNow upload it here:")

uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
print("\n" + "=" * 60)
print("STEP 2: Downloading dataset from Kaggle...")
print("=" * 60)

!kaggle datasets download -d patelris/crop-yield-prediction-dataset
!unzip -o crop-yield-prediction-dataset.zip

# Load the dataset
df = pd.read_csv('yield_df.csv')

print("\n✓ Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

STEP 1: Upload your kaggle.json file
Go to: https://www.kaggle.com/settings/account
Scroll down to 'API' section and click 'Create New Token'
This will download kaggle.json

Now upload it here:


Saving kaggle.json to kaggle.json

STEP 2: Downloading dataset from Kaggle...
Dataset URL: https://www.kaggle.com/datasets/patelris/crop-yield-prediction-dataset
License(s): world-bank
  0% 0.00/959k [00:00<?, ?B/s]
100% 959k/959k [00:00<00:00, 586MB/s]
Archive:  crop-yield-prediction-dataset.zip
  inflating: pesticides.csv          
  inflating: rainfall.csv            
  inflating: temp.csv                
  inflating: yield.csv               
  inflating: yield_df.csv            

✓ Dataset loaded successfully!
Dataset shape: (28242, 8)

Columns: ['Unnamed: 0', 'Area', 'Item', 'Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

First few rows:
   Unnamed: 0     Area         Item  Year  hg/ha_yield  \
0           0  Albania        Maize  1990        36613   
1           1  Albania     Potatoes  1990        66667   
2           2  Albania  Rice, paddy  1990        23333   
3           3  Albania      Sorghum  1990        12500   
4           4  Al

In [3]:
# ============================================================================
# PART 3: DATA PREPROCESSING AND SCHEMA DESIGN
# ============================================================================

print("\n" + "=" * 60)
print("STEP 3: Data Preprocessing")
print("=" * 60)

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Handle missing values
df = df.dropna()

# Display basic statistics
print(f"\nDataset Info:")
print(f"Total records: {len(df)}")
print(f"\nColumns and types:")
print(df.dtypes)
print(f"\nBasic statistics:")
print(df.describe())


STEP 3: Data Preprocessing

Dataset Info:
Total records: 28242

Columns and types:
unnamed:_0                         int64
area                              object
item                              object
year                               int64
hg/ha_yield                        int64
average_rain_fall_mm_per_year    float64
pesticides_tonnes                float64
avg_temp                         float64
dtype: object

Basic statistics:
         unnamed:_0          year    hg/ha_yield  \
count  28242.000000  28242.000000   28242.000000   
mean   14120.500000   2001.544296   77053.332094   
std     8152.907488      7.051905   84956.612897   
min        0.000000   1990.000000      50.000000   
25%     7060.250000   1995.000000   19919.250000   
50%    14120.500000   2001.000000   38295.000000   
75%    21180.750000   2008.000000  104676.750000   
max    28241.000000   2013.000000  501412.000000   

       average_rain_fall_mm_per_year  pesticides_tonnes      avg_temp  
count         

In [4]:
# ============================================================================
# PART 4: CREATE RELATIONAL DATABASE (SQLite)
# ============================================================================

print("\n" + "=" * 60)
print("STEP 4: Creating Relational Database (SQLite)")
print("=" * 60)

import sqlite3
from datetime import datetime

# Create connection
conn = sqlite3.connect('crop_yield.db')
cursor = conn.cursor()

# Drop existing tables if they exist
cursor.execute("DROP TABLE IF EXISTS yield_records")
cursor.execute("DROP TABLE IF EXISTS crops")
cursor.execute("DROP TABLE IF EXISTS countries")
cursor.execute("DROP TABLE IF EXISTS audit_log")

# Create Countries Table
cursor.execute("""
CREATE TABLE countries (
    country_id INTEGER PRIMARY KEY AUTOINCREMENT,
    country_name VARCHAR(100) UNIQUE NOT NULL,
    region VARCHAR(50),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
# Create Crops Table
cursor.execute("""
CREATE TABLE crops (
    crop_id INTEGER PRIMARY KEY AUTOINCREMENT,
    crop_name VARCHAR(100) UNIQUE NOT NULL,
    crop_category VARCHAR(50),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Create Yield Records Table
cursor.execute("""
CREATE TABLE yield_records (
    record_id INTEGER PRIMARY KEY AUTOINCREMENT,
    country_id INTEGER NOT NULL,
    crop_id INTEGER NOT NULL,
    year INTEGER NOT NULL,
    avg_temp REAL,
    avg_rainfall_mm REAL,
    pesticides_tonnes REAL,
    yield_hg_per_ha REAL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (country_id) REFERENCES countries(country_id),
    FOREIGN KEY (crop_id) REFERENCES crops(crop_id),
    CHECK (year >= 1900 AND year <= 2100),
    CHECK (yield_hg_per_ha >= 0)
)
""")

# Create Audit Log Table
cursor.execute("""
CREATE TABLE audit_log (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    table_name VARCHAR(50),
    operation VARCHAR(20),
    record_id INTEGER,
    old_value TEXT,
    new_value TEXT,
    changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

print("✓ Tables created successfully!")


STEP 4: Creating Relational Database (SQLite)
✓ Tables created successfully!


In [5]:
# ============================================================================
# PART 5: CREATE TRIGGER
# ============================================================================

print("\n" + "=" * 60)
print("STEP 5: Creating Trigger")
print("=" * 60)

# Trigger to log updates to yield_records
cursor.execute("""
CREATE TRIGGER log_yield_updates
AFTER UPDATE ON yield_records
FOR EACH ROW
BEGIN
    INSERT INTO audit_log (table_name, operation, record_id, old_value, new_value)
    VALUES (
        'yield_records',
        'UPDATE',
        NEW.record_id,
        json_object(
            'yield', OLD.yield_hg_per_ha,
            'rainfall', OLD.avg_rainfall_mm,
            'temp', OLD.avg_temp
        ),
        json_object(
            'yield', NEW.yield_hg_per_ha,
            'rainfall', NEW.avg_rainfall_mm,
            'temp', NEW.avg_temp
        )
    );
END;
""")

# Trigger to automatically update updated_at timestamp
cursor.execute("""
CREATE TRIGGER update_timestamp
AFTER UPDATE ON yield_records
FOR EACH ROW
BEGIN
    UPDATE yield_records
    SET updated_at = CURRENT_TIMESTAMP
    WHERE record_id = NEW.record_id;
END;
""")

print("✓ Triggers created successfully!")
print("  - log_yield_updates: Logs all updates to yield_records")
print("  - update_timestamp: Auto-updates timestamp on modifications")



STEP 5: Creating Trigger
✓ Triggers created successfully!
  - log_yield_updates: Logs all updates to yield_records
  - update_timestamp: Auto-updates timestamp on modifications


In [6]:
# ============================================================================
# PART 6: CREATE STORED PROCEDURES (as Functions)
# ============================================================================

print("\n" + "=" * 60)
print("STEP 6: Creating Stored Procedures (Functions)")
print("=" * 60)

# Note: SQLite doesn't support stored procedures natively,
# so we'll create Python functions that perform the same operations

def get_crop_statistics(crop_name):
    """
    Stored Procedure: Get statistics for a specific crop
    """
    query = """
    SELECT
        c.crop_name,
        COUNT(*) as total_records,
        AVG(yr.yield_hg_per_ha) as avg_yield,
        MIN(yr.yield_hg_per_ha) as min_yield,
        MAX(yr.yield_hg_per_ha) as max_yield,
        AVG(yr.avg_temp) as avg_temperature,
        AVG(yr.avg_rainfall_mm) as avg_rainfall
    FROM yield_records yr
    JOIN crops c ON yr.crop_id = c.crop_id
    WHERE c.crop_name = ?
    GROUP BY c.crop_name
    """
    cursor.execute(query, (crop_name,))
    result = cursor.fetchone()

    if result:
        return {
            'crop_name': result[0],
            'total_records': result[1],
            'avg_yield': round(result[2], 2),
            'min_yield': round(result[3], 2),
            'max_yield': round(result[4], 2),
            'avg_temperature': round(result[5], 2),
            'avg_rainfall': round(result[6], 2)
        }
    return None

def validate_and_insert_yield_record(country_name, crop_name, year,
                                     avg_temp, avg_rainfall, pesticides, yield_val):
    """
    Stored Procedure: Validate and insert a new yield record
    """
    try:
        # Validate inputs
        if year < 1900 or year > 2100:
            return False, "Invalid year"
        if yield_val < 0:
            return False, "Yield cannot be negative"

        # Get or create country
        cursor.execute("SELECT country_id FROM countries WHERE country_name = ?", (country_name,))
        country_row = cursor.fetchone()
        if country_row:
            country_id = country_row[0]
        else:
            cursor.execute("INSERT INTO countries (country_name) VALUES (?)", (country_name,))
            country_id = cursor.lastrowid

        # Get or create crop
        cursor.execute("SELECT crop_id FROM crops WHERE crop_name = ?", (crop_name,))
        crop_row = cursor.fetchone()
        if crop_row:
            crop_id = crop_row[0]
        else:
            cursor.execute("INSERT INTO crops (crop_name) VALUES (?)", (crop_name,))
            crop_id = cursor.lastrowid

        # Insert yield record
        cursor.execute("""
            INSERT INTO yield_records
            (country_id, crop_id, year, avg_temp, avg_rainfall_mm, pesticides_tonnes, yield_hg_per_ha)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (country_id, crop_id, year, avg_temp, avg_rainfall, pesticides, yield_val))

        conn.commit()
        return True, "Record inserted successfully"

    except Exception as e:
        return False, str(e)

print("✓ Stored Procedures (Functions) created successfully!")
print("  - get_crop_statistics(crop_name)")
print("  - validate_and_insert_yield_record(...)")


STEP 6: Creating Stored Procedures (Functions)
✓ Stored Procedures (Functions) created successfully!
  - get_crop_statistics(crop_name)
  - validate_and_insert_yield_record(...)


In [ ]:
# ============================================================================
# PART 7: POPULATE DATABASE (FINAL CORRECTED VERSION)
# ============================================================================

print("\n" + "=" * 60)
print("STEP 7: Populating Database (Final Corrected Version)")
print("=" * 60)

# Let's use the actual column names from the DataFrame
df_working = df.copy()

print("Actual columns in DataFrame:")
for i, col in enumerate(df_working.columns, 1):
    print(f"  {i}. '{col}'")

# Create a flexible column mapping based on actual column names
column_mapping = {
    'country_name': 'country_name',
    'crop_name': 'crop_name',
    'year': 'year',
    'yield_hg_per_ha': 'yield_hg_per_ha',
    'rainfall_mm': 'avg_rainfall_mm',  # This is the actual column name
    'pesticides_tonnes': 'pesticides_tonnes',
    'avg_temp': 'avg_temp'
}

# Verify all required columns exist
required_columns = ['country_name', 'crop_name', 'year', 'yield_hg_per_ha', 'avg_rainfall_mm', 'pesticides_tonnes', 'avg_temp']
missing_columns = [col for col in required_columns if col not in df_working.columns]

if missing_columns:
    print(f"\n⚠ Missing columns: {missing_columns}")
    print("Available columns:")
    for col in df_working.columns:
        print(f"  - '{col}'")
else:
    print(f"\n✓ All required columns found!")

# Clear existing data to start fresh
print("\nClearing existing data...")
cursor.execute("DELETE FROM yield_records")
cursor.execute("DELETE FROM countries")
cursor.execute("DELETE FROM crops")
cursor.execute("DELETE FROM audit_log")
conn.commit()
print("✓ Cleared existing data")

# Insert unique countries
print("\n1. Inserting countries...")
unique_countries = df_working['area'].unique() # Changed from country_name to area
print(f"Found {len(unique_countries)} unique countries")

countries_inserted = 0
for country in unique_countries:
    try:
        cursor.execute("INSERT INTO countries (country_name) VALUES (?)", (country,))
        countries_inserted += 1
    except Exception as e:
        # Ignore duplicate errors due to UNIQUE constraint
        if "UNIQUE constraint" not in str(e):
            print(f"Error inserting country '{country}': {e}")

conn.commit()
print(f"✓ Inserted {countries_inserted} countries")

# Insert unique crops
print("\n2. Inserting crops...")
unique_crops = df_working['item'].unique() # Changed from crop_name to item
print(f"Found {len(unique_crops)} unique crops")

crops_inserted = 0
for crop in unique_crops:
    try:
        cursor.execute("INSERT INTO crops (crop_name) VALUES (?)", (crop,))
        crops_inserted += 1
    except Exception as e:
        # Ignore duplicate errors due to UNIQUE constraint
        if "UNIQUE constraint" not in str(e):
            print(f"Error inserting crop '{crop}': {e}")

conn.commit()
print(f"✓ Inserted {crops_inserted} crops")

# Verify country and crop IDs were created
cursor.execute("SELECT COUNT(*) FROM countries")
country_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM crops")
crop_count = cursor.fetchone()[0]
print(f"\nDatabase now has {country_count} countries and {crop_count} crops")

# Insert yield records - USING ACTUAL COLUMN NAMES
print("\n3. Inserting yield records...")
records_inserted = 0
errors = 0

# Create a mapping dictionary for country and crop names to IDs
cursor.execute("SELECT country_id, country_name FROM countries")
country_map = {name: cid for cid, name in cursor.fetchall()}

cursor.execute("SELECT crop_id, crop_name FROM crops")
crop_map = {name: cid for cid, name in cursor.fetchall()}

# Insert records in batches for better performance
batch_size = 1000
batch_records = []

for idx, row in df_working.iterrows():
    try:
        # Use ACTUAL column names from the DataFrame
        country_name = row['area'] # Changed from country_name to area
        crop_name = row['item'] # Changed from crop_name to item
        year = int(row['year'])
        yield_value = float(row['hg/ha_yield'])
        rainfall_mm = float(row['average_rain_fall_mm_per_year'])  # This is the correct column name
        pesticides_tonnes = float(row['pesticides_tonnes'])
        avg_temp = float(row['avg_temp'])

        # Get IDs from our mapping
        country_id = country_map.get(country_name)
        crop_id = crop_map.get(crop_name)

        if not country_id:
            print(f"Warning: Country '{country_name}' not found in mapping")
            errors += 1
            continue

        if not crop_id:
            print(f"Warning: Crop '{crop_name}' not found in mapping")
            errors += 1
            continue

        # Add to batch
        batch_records.append((
            country_id, crop_id, year, avg_temp, rainfall_mm,
            pesticides_tonnes, yield_value
        ))

        # Insert batch when it reaches batch_size
        if len(batch_records) >= batch_size:
            cursor.executemany("""
                INSERT INTO yield_records
                (country_id, crop_id, year, avg_temp, avg_rainfall_mm, pesticides_tonnes, yield_hg_per_ha)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, batch_records)
            records_inserted += len(batch_records)
            batch_records = []
            print(f"  Inserted {records_inserted} records...")

    except Exception as e:
        errors += 1
        if errors <= 3:  # Only show first few errors
            print(f"Error at row {idx}: {e}")
            print(f"  Row data: {dict(row)}")
        continue

# Insert any remaining records in the final batch
if batch_records:
    cursor.executemany("""
        INSERT INTO yield_records
        (country_id, crop_id, year, avg_temp, avg_rainfall_mm, pesticides_tonnes, yield_hg_per_ha)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, batch_records)
    records_inserted += len(batch_records)

conn.commit()

print(f"\n✓ Insertion completed!")
print(f"  Successfully inserted: {records_inserted} records")
print(f"  Errors encountered: {errors} records")
print(f"  Total processed: {records_inserted + errors} records")

# Verify the insertion
cursor.execute("SELECT COUNT(*) FROM yield_records")
final_count = cursor.fetchone()[0]
print(f"✓ Database now contains {final_count} yield records")

# Show sample of inserted data
if final_count > 0:
    print("\nSample of inserted data:")
    cursor.execute("""
        SELECT c.country_name, cr.crop_name, yr.year, yr.yield_hg_per_ha
        FROM yield_records yr
        JOIN countries c ON yr.country_id = c.country_id
        JOIN crops cr ON yr.crop_id = cr.crop_id
        LIMIT 5
    """)
    sample_data = cursor.fetchall()
    for row in sample_data:
        print(f"  {row[0]} - {row[1]} ({row[2]}): {row[3]:,.0f} hg/ha")

    # Show some statistics
    print(f"\nDatabase Statistics:")
    cursor.execute("SELECT MIN(year), MAX(year) FROM yield_records")
    min_year, max_year = cursor.fetchone()
    print(f"  Year range: {min_year} - {max_year}")

    cursor.execute("SELECT COUNT(DISTINCT country_id) FROM yield_records")
    unique_countries = cursor.fetchone()[0]
    print(f"  Countries with data: {unique_countries}")

    cursor.execute("SELECT COUNT(DISTINCT crop_id) FROM yield_records")
    unique_crops = cursor.fetchone()[0]
    print(f"  Crops with data: {unique_crops}")
else:
    print("\n⚠ No records were inserted. Debugging info:")
    print(f"  Total rows in DataFrame: {len(df_working)}")
    print(f"  First row sample: {dict(df_working.iloc[0])}")


STEP 7: Populating Database (Final Corrected Version)
Actual columns in DataFrame:
  1. 'unnamed:_0'
  2. 'area'
  3. 'item'
  4. 'year'
  5. 'hg/ha_yield'
  6. 'average_rain_fall_mm_per_year'
  7. 'pesticides_tonnes'
  8. 'avg_temp'

⚠ Missing columns: ['country_name', 'crop_name', 'yield_hg_per_ha', 'avg_rainfall_mm']
Available columns:
  - 'unnamed:_0'
  - 'area'
  - 'item'
  - 'year'
  - 'hg/ha_yield'
  - 'average_rain_fall_mm_per_year'
  - 'pesticides_tonnes'
  - 'avg_temp'

Clearing existing data...
✓ Cleared existing data

1. Inserting countries...
Found 101 unique countries
✓ Inserted 101 countries

2. Inserting crops...
Found 10 unique crops
✓ Inserted 10 crops

Database now has 101 countries and 10 crops

3. Inserting yield records...
  Inserted 1000 records...
  Inserted 2000 records...
  Inserted 3000 records...
  Inserted 4000 records...
  Inserted 5000 records...
  Inserted 6000 records...
  Inserted 7000 records...
  Inserted 8000 records...
  Inserted 9000 records...
 

In [7]:
# ============================================================================
# PART 8: TEST STORED PROCEDURES AND TRIGGERS
# ============================================================================

print("\n" + "=" * 60)
print("STEP 8: Testing Stored Procedures and Triggers")
print("=" * 60)

# Test stored procedure - get_crop_statistics
print("\n1. Testing get_crop_statistics()...")
cursor.execute("SELECT crop_name FROM crops LIMIT 3")
sample_crops = cursor.fetchall()

for crop_row in sample_crops:
    crop_name = crop_row[0]
    stats = get_crop_statistics(crop_name)
    if stats:
        print(f"\n Statistics for {crop_name}:")
        print(f"   Total Records: {stats['total_records']}")
        print(f"   Average Yield: {stats['avg_yield']:,.2f} hg/ha")
        print(f"   Min Yield: {stats['min_yield']:,.2f} hg/ha")
        print(f"   Max Yield: {stats['max_yield']:,.2f} hg/ha")
        print(f"   Avg Temperature: {stats['avg_temperature']:.2f}°C")
        print(f"   Avg Rainfall: {stats['avg_rainfall']:.2f} mm")

# Test the insert stored procedure
print("\n2. Testing validate_and_insert_yield_record()...")
test_cases = [
    # Valid case
    ("Test Country", "Test Crop", 2020, 20.5, 1000.0, 500.0, 50000.0),
    # Invalid year
    ("Test Country", "Test Crop", 1800, 20.5, 1000.0, 500.0, 50000.0),
    # Negative yield
    ("Test Country", "Test Crop", 2020, 20.5, 1000.0, 500.0, -100.0)
]

for i, test_case in enumerate(test_cases, 1):
    success, message = validate_and_insert_yield_record(*test_case)
    status = "✓ SUCCESS" if success else "✗ FAILED"
    print(f"   Test {i}: {status} - {message}")

# Test trigger by updating a record
print("\n3. Testing trigger (updating a record)...")
cursor.execute("""
    SELECT record_id, yield_hg_per_ha, avg_temp, avg_rainfall_mm
    FROM yield_records
    LIMIT 1
""")
test_record = cursor.fetchone()

if test_record:
    record_id, old_yield, old_temp, old_rainfall = test_record
    new_yield = old_yield * 1.15  # Increase by 15%
    new_temp = old_temp + 1.0     # Increase temperature by 1°C

    print(f"   Updating record {record_id}:")
    print(f"     Yield: {old_yield:,.0f} → {new_yield:,.0f} hg/ha")
    print(f"     Temperature: {old_temp:.1f} → {new_temp:.1f}°C")

    cursor.execute("""
        UPDATE yield_records
        SET yield_hg_per_ha = ?, avg_temp = ?
        WHERE record_id = ?
    """, (new_yield, new_temp, record_id))
    conn.commit()

    # Check audit log
    cursor.execute("SELECT * FROM audit_log ORDER BY log_id DESC LIMIT 1")
    audit_entry = cursor.fetchone()

    if audit_entry:
        print(f"\n   ✓ Audit Log Entry Created:")
        print(f"     Log ID: {audit_entry[0]}")
        print(f"     Table: {audit_entry[1]}")
        print(f"     Operation: {audit_entry[2]}")
        print(f"     Record ID: {audit_entry[3]}")
        print(f"     Changed At: {audit_entry[6]}")

        # Parse JSON data
        import json
        try:
            old_data = json.loads(audit_entry[4]) if audit_entry[4] else {}
            new_data = json.loads(audit_entry[5]) if audit_entry[5] else {}
            print(f"     Old Values: Yield={old_data.get('yield', 'N/A')}, Temp={old_data.get('temp', 'N/A')}")
            print(f"     New Values: Yield={new_data.get('yield', 'N/A')}, Temp={new_data.get('temp', 'N/A')}")
        except Exception as e:
            print(f"     JSON Parse Error: {e}")
    else:
        print("   ⚠ No audit log entry found")
else:
    print("   ⚠ No records found to test trigger")

# Test updated_at trigger
print("\n4. Testing updated_at timestamp trigger...")
cursor.execute("SELECT record_id, updated_at FROM yield_records LIMIT 1")
record_before = cursor.fetchone()

if record_before:
    record_id, old_updated_at = record_before
    cursor.execute("""
        UPDATE yield_records
        SET yield_hg_per_ha = yield_hg_per_ha + 1000
        WHERE record_id = ?
    """, (record_id,))
    conn.commit()

    cursor.execute("SELECT updated_at FROM yield_records WHERE record_id = ?", (record_id,))
    new_updated_at = cursor.fetchone()[0]

    print(f"   Record {record_id}:")
    print(f"     Before update: {old_updated_at}")
    print(f"     After update:  {new_updated_at}")
    print(f"     ✓ Timestamp updated: {old_updated_at != new_updated_at}")


STEP 8: Testing Stored Procedures and Triggers

1. Testing get_crop_statistics()...

2. Testing validate_and_insert_yield_record()...
   Test 1: ✓ SUCCESS - Record inserted successfully
   Test 2: ✗ FAILED - Invalid year
   Test 3: ✗ FAILED - Yield cannot be negative

3. Testing trigger (updating a record)...
   Updating record 1:
     Yield: 50,000 → 57,500 hg/ha
     Temperature: 20.5 → 21.5°C

   ✓ Audit Log Entry Created:
     Log ID: 2
     Table: yield_records
     Operation: UPDATE
     Record ID: 1
     Changed At: 2025-11-02 18:34:05
     Old Values: Yield=50000.0, Temp=20.5
     New Values: Yield=57500.0, Temp=21.5

4. Testing updated_at timestamp trigger...
   Record 1:
     Before update: 2025-11-02 18:34:05
     After update:  2025-11-02 18:34:05
     ✓ Timestamp updated: False
